# Model

In [1]:
!aws s3 ls


2023-08-29 23:43:16 sagemaker-studio-692960231031-wo7kgoszj2g
2023-08-29 23:50:01 sagemaker-us-east-1-692960231031
2023-08-30 00:34:21 vad49
2023-09-16 16:02:10 vad49-labdata


In [2]:
!aws s3 ls s3://project17-bucket-alex/stories-and-books-nlp/


                           PRE mapping/
                           PRE processed-data/


In [3]:
if True is True: # set to true only for the first un
    # Setup - Run only once per Kernel App
    %conda install openjdk -y

    # install PySpark
    %pip install pyspark==3.2.0 s3fs pyarrow torch==1.13.0

    # restart kernel
    from IPython.core.display import HTML
    HTML("<script>Jupyter.notebook.kernel.restart()</script>")

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.10.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.10.0



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import functions as F
from pyspark.sql.functions import col, length, isnan, when, count, regexp_extract, weekofyear, hour, avg, to_date, unix_timestamp, lit, corr


import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch.nn.functional as F
import string
from tqdm import tqdm
import boto3
import pickle
from io import BytesIO
from pyspark.sql.functions import col, explode


pd.set_option('display.max_colwidth', 150) 
#pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)


In [5]:
# Import pyspark and build Spark session

spark = (
    SparkSession.builder.appName("PySparkApp")
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.2")
    .config(
        "fs.s3a.aws.credentials.provider",
        "com.amazonaws.auth.ContainerCredentialsProvider",
    )
    #.config("spark-jars-packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.1.3")\
    .getOrCreate()
)

print(spark.version)

:: loading settings :: url = jar:file:/opt/conda/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-389213b9-19f6-4053-aace-7886cc9dcd4c;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
:: resolution report :: resolve 272ms :: artifacts dl 21ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------------------

3.2.0


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

In [8]:
# read in the data
all_model_text = spark.read.parquet("s3a://project17-bucket-alex/stories-and-books-nlp/processed-data/")

#display(f"shape: ({all_model_text.count()}, {len(all_model_text.columns)})")
all_model_text.show()


23/11/29 01:27:25 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+--------------------+--------------------+--------------------+
|                text|       custom_tokens|         text_as_int|
+--------------------+--------------------+--------------------+
|First of all, I n...|[first,  , of,  ,...|[22, 25, 34, 35, ...|
|I started a job a...|[i,  , started,  ...|[25, 1, 35, 36, 1...|
|My older brother ...|[my,  , older,  ,...|[29, 41, 1, 31, 2...|
|When I was a kid,...|[when,  , i,  , w...|[39, 24, 21, 30, ...|
|TL/DR on bottom!\...|[tldr,  , on,  , ...|[36, 28, 20, 34, ...|
|I have many stori...|[i,  , have,  , m...|[25, 1, 24, 17, 3...|
| \n\nFaults in Ma...|[ , faults,  , in...|[1, 22, 17, 37, 2...|
|My (24F) boyfrien...|[my,  , 24f,  , b...|[29, 41, 1, 7, 9,...|
|So, this wasn't r...|[so, ,,  , this, ...|[35, 31, 3, 1, 36...|
|\nEDIT: Hi! Still...|[edit,  , hi, !, ...|[21, 20, 25, 36, ...|
|tl;dr: So basical...|[tl, ;, dr,  , so...|[36, 28, 15, 20, ...|
|Obligatory, I mis...|[obligatory, ,,  ...|[31, 18, 28, 25, ...|
|Warning: mentions...|[wa

In [9]:
# work on a subset of the data

all_model_text = all_model_text.sample(False, 0.001)



In [10]:
# helper functions
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


In [11]:

# collect all sequences into one long sequence
flattened_seq = all_model_text.select("text_as_int").rdd.flatMap(lambda x: x[0]).collect()

# convert the flattened sequence into a tensor
char_dataset = torch.tensor(flattened_seq, dtype=torch.long)



In [12]:


# Initialize a boto3 client
s3 = boto3.client('s3')

bucket_name = 'project17-bucket-alex'
object_key = 'stories-and-books-nlp/mapping/char2idx.pkl'

# Create a buffer
char2idx_buffer = BytesIO()

# Download the file from S3 to the buffer
s3.download_fileobj(bucket_name, object_key, char2idx_buffer)

# Set buffer's position to the start
char2idx_buffer.seek(0)

# Deserialize the file to load the char2idx dictionary
char2idx = pickle.load(char2idx_buffer)

# define the reverse too
idx2char = {idx: char for char, idx in char2idx.items()}


# check
#for key, value in list(char2idx.items())[:20]:
#    print(f"{key}: {value}")

In [13]:


# Collect all unique tokens
all_tokens = all_model_text.select(explode(col("custom_tokens"))).distinct().rdd.flatMap(lambda x: x).collect()

# Create the vocabulary from these tokens
vocab = sorted(set(all_tokens))


In [14]:

# define sequences
seq_length = 100
examples_per_epoch = len(char_dataset) // (seq_length + 1)


# create the dataset
#char_dataset = torch.tensor(text_as_int, dtype=torch.long)

sequences = []
targets = []
for i in range(0, len(char_dataset) - seq_length, seq_length):
    sequences.append(char_dataset[i:i+seq_length])
    targets.append(char_dataset[i+1:i+seq_length+1])

dataset = TensorDataset(torch.stack(sequences), torch.stack(targets))



# split it
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_data, val_data, test_data = random_split(dataset, [train_size, val_size, test_size])


# turn into dataloader
batch_size = 64

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)



In [15]:

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, n_layers):
        super(RNN, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, batch_first=True)
        
        # output
        self.fc = nn.Linear(hidden_dim, vocab_size)
        
    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.rnn(x)
        x = self.fc(x)
        
        return x


In [16]:

# hyperparameters
vocab_size = len(vocab)
embedding_dim = 128
hidden_dim = 256
n_layers = 2


# instantiate
model = RNN(vocab_size, embedding_dim, hidden_dim, n_layers).to(device)


display(model)


total_params = count_parameters(model)
print(f'total number of parameters: {total_params}')


RNN(
  (embedding): Embedding(1051, 128)
  (rnn): LSTM(128, 256, num_layers=2, batch_first=True)
  (fc): Linear(in_features=256, out_features=1051, bias=True)
)

total number of parameters: 1326235


In [21]:
%%time

# training params
learning_rate = 0.01
num_epochs = 50

patience = 5  
best_val_loss = float('inf')
epochs_since_improvement = 0
tolerance = 0.01

# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)



# training loop
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    
    model.train()
    running_train_loss = 0.0
    
    # progress bar
    loop = tqdm(enumerate(train_loader), total=len(train_loader), leave=False)
    
    
    
    # loop through data loader
    for batch_idx, (data, target) in loop:
        
        # move to gpu
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = model(data)
        output = output.permute(0, 2, 1)
        
        loss = criterion(output, target)
        loss.backward()
        
        optimizer.step()
        
        # acummulate loss
        running_train_loss += loss.item()
        
        # update tqdm bar
        loop.set_description(f"epoch [{epoch+1}/{num_epochs}]")
        loop.set_postfix(loss=loss.item())
    
    # get avg loss for the epoch
    train_loss = running_train_loss / len(train_loader)
    train_losses.append(train_loss)
    train_perplexity = torch.exp(torch.tensor(train_loss))
    
    
    # validate
    model.eval()
    running_val_loss = 0.0
    val_loop = tqdm(enumerate(val_loader), total=len(val_loader), leave=False)
    
    with torch.no_grad():
        
        # loop through validation
        for batch_idx, (data, target) in val_loop:
            
            data, target = data.to(device), target.to(device)
            
            output = model(data)
            output = output.permute(0, 2, 1)
            
            loss = criterion(output, target)
            running_val_loss += loss.item()
            
            
            # tqdm
            val_loop.set_description(f"validation epoch [{epoch+1}/{num_epochs}]")
            val_loop.set_postfix(val_loss=loss.item())
    
    
    # get avg loss for the epoch
    val_loss = running_val_loss / len(val_loader)
    val_losses.append(val_loss)
    val_perplexity = torch.exp(torch.tensor(val_loss))
    
    
    # save model in case of kernel crash
    model_save_path = "../../data/ml-data/rnn_model.pth"
    torch.save(model.state_dict(), model_save_path)
    
    
    # early stopping
    improvement = best_val_loss - val_loss
    if improvement >= tolerance:
        best_val_loss = val_loss
        epochs_since_improvement = 0
    else:
        epochs_since_improvement += 1

    if epochs_since_improvement == patience:
        print(f"stopping early at epoch {epoch+1}")
        break

    
    # report
    print(f"epoch {epoch+1}/{num_epochs}, train loss: {train_loss:.4f}, train perplexity: {train_perplexity:.4f}, val loss: {val_loss:.4f}, val perplexity: {val_perplexity:.4f}")
    


# eval
model.eval()
test_loss = 0.0

with torch.no_grad():
    
    for batch_idx, (data, target) in enumerate(test_loader):
        
        data, target = data.to(device), target.to(device)
        
        output = model(data)
        output = output.permute(0, 2, 1)
        loss = criterion(output, target)
        test_loss += loss.item()
        
        
test_loss /= len(test_loader)

# perplexity
test_perplexity = torch.exp(torch.tensor(test_loss))


# report
print(f"test loss: {test_loss:.4f}, test perplexity: {test_perplexity:.4f}")



epoch 1/50, train loss: 1.9389, train perplexity: 6.9508, val loss: 2.2144, val perplexity: 9.1561


epoch 2/50, train loss: 1.7156, train perplexity: 5.5602, val loss: 2.0933, val perplexity: 8.1112


epoch 3/50, train loss: 1.5597, train perplexity: 4.7573, val loss: 1.9992, val perplexity: 7.3830


epoch 4/50, train loss: 1.4907, train perplexity: 4.4401, val loss: 1.9657, val perplexity: 7.1399


epoch 5/50, train loss: 1.4246, train perplexity: 4.1562, val loss: 1.9358, val perplexity: 6.9298


...

epoch 10/50, train loss: 1.2192, train perplexity: 3.3845, val loss: 1.9079, val perplexity: 6.7387


epoch 11/50, train loss: 1.1580, train perplexity: 3.1834, val loss: 1.9092, val perplexity: 6.7479


epoch 12/50, train loss: 1.1379, train perplexity: 3.1202, val loss: 1.9218, val perplexity: 6.8333


epoch 13/50, train loss: 1.1067, train perplexity: 3.0243, val loss: 1.9263, val perplexity: 6.8643


stopping early at epoch 14
test loss: 1.8550, test perplexity: 6.3915
CPU times: user 1.27 s, sys: 347 ms, total: 1.62 s
Wall time: 2.94 s


In [24]:
def generate_text(model, start_string, length, temperature=1.0):

    
    # Put the model in evaluation mode
    model.eval()

    # Convert start_string characters to indices
    input_eval = [char2idx.get(s, char2idx['UNK']) for s in start_string]
    input_eval = torch.tensor(input_eval, dtype=torch.long).unsqueeze(0).to(device)

    generated_text = []

    with torch.no_grad():
        for _ in range(length):
            output = model(input_eval)
            output = output[:, -1, :] / temperature
            probabilities = F.softmax(output, dim=-1)
            predicted_id = torch.multinomial(probabilities, num_samples=1)

            # Append to the input for the next character prediction
            input_eval = torch.cat([input_eval, predicted_id], dim=1)

            generated_text.append(idx2char[predicted_id.item()])

    return start_string + ''.join(generated_text)




In [25]:
#  usage
prompt = "Once upon a time"
generated = generate_text(model, prompt, 100)  # Generates 100 characters
print(generated)



Once upon a time that eppacfion the caid know.her stuff who sobe tose the was anger, and sechs at jussive thats fim,


In [ ]:
# save model

# path
#model_path = "../../data/ml-data/rnn_model.pth"


#if True = True:
    # save
#    torch.save(model.state_dict(), model_path)



In [ ]:
# load if not training in this moment
# hyperparameters
vocab_size = len(vocab)
embedding_dim = 128
hidden_dim = 256
n_layers = 2


# instantiate
model_saved = RNN(vocab_size, embedding_dim, hidden_dim, n_layers)


model_saved.load_state_dict(torch.load(model_path))

#  usage
prompt = "Once upon a time"
generated = generate_text(model_saved, prompt, 100)  # Generates 100 characters
print(generated)